<a href="https://colab.research.google.com/github/iashutosh99/CODSOFT_1/blob/main/Movie_ds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import dependencies

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from scipy.sparse import hstack
import numpy as np

In [ ]:
# Load the  dataset with a different encoding
movies = pd.read_csv('/content/drive/MyDrive/ Dataset/IMDb Movies India.csv', encoding='ISO-8859-1')


In [ ]:
#Print 5 rows of the datag

,Name,Year,Duration,Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3
0,,NaN,NaN,Drama,NaN,NaN,J.S. Randhawa,Manmauji,Birbal,Rajendra Bhatia
1,#Gadhvi (He thought he was Gandhi),(2019),109 min,Drama,7.0,8,Gaurav Bakshi,Rasika Dugal,Vivek Ghamande,Arvind Jangid
2,#Homecoming,(2021),90 min,"Drama, Musical",NaN,NaN,Soumyajit Majumdar,Sayani Gupta,Plabita Borthakur,Roy Angana
3,#Yaaram,(2019),110 min,"Comedy, Romance",4.4,35,Ovais Khan,Prateik,Ishita Raj,Siddhant Kapoor
4,...And Once Again,(2010),105 min,Drama,NaN,NaN,Amol Palekar,Rajat Kapoor,Rituparna Sengupta,Antara Mali


In [ ]:
#Print last 5 rows of the data
movies.tail(5)

,Name,Year,Duration,Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3
15504,Zulm Ko Jala Doonga,(1988),NaN,Action,4.6,11,Mahendra Shah,Naseeruddin Shah,Sumeet Saigal,Suparna Anand
15505,Zulmi,(1999),129 min,"Action, Drama",4.5,655,Kuku Kohli,Akshay Kumar,Twinkle Khanna,Aruna Irani
15506,Zulmi Raj,(2005),NaN,Action,NaN,NaN,Kiran Thej,Sangeeta Tiwari,NaN,NaN
15507,Zulmi Shikari,(1988),NaN,Action,NaN,NaN,NaN,NaN,NaN,NaN
15508,Zulm-O-Sitam,(1998),130 min,"Action, Drama",6.2,20,K.C. Bokadia,Dharmendra,Jaya Prada,Arjun Sarja


In [ ]:
#Get some information about dataset
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15509 entries, 0 to 15508
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Name      15509 non-null  object 
 1   Year      14981 non-null  object 
 2   Duration  7240 non-null   object 
 3   Genre     13632 non-null  object 
 4   Rating    7919 non-null   float64
 5   Votes     7920 non-null   object 
 6   Director  14984 non-null  object 
 7   Actor 1   13892 non-null  object 
 8   Actor 2   13125 non-null  object 
 9   Actor 3   12365 non-null  object 
dtypes: float64(1), object(9)
memory usage: 1.2+ MB


In [ ]:
#Checking the number of missingvalues in each column
movies.isnull().sum()

Name           0
Year         528
Duration    8269
Genre       1877
Rating      7590
Votes       7589
Director     525
Actor 1     1617
Actor 2     2384
Actor 3     3144
dtype: int64

In [ ]:
# Data Preprocessing
# Fill missing values for text columns
movies['Genre'].fillna('', inplace=True)
movies['Director'].fillna('', inplace=True)
movies['Actor 1'].fillna('', inplace=True)
movies['Actor 2'].fillna('', inplace=True)
movies['Actor 3'].fillna('', inplace=True)


In [ ]:
# Ensure all entries are treated as strings before cleaning
movies['Year'] = movies['Year'].astype(str)
movies['Duration'] = movies['Duration'].astype(str)
movies['Votes'] = movies['Votes'].astype(str)

In [ ]:
# Clean and convert numerical columns
# Extract numeric parts from strings, replace non-numeric characters, and convert to float
movies['Year'] = movies['Year'].str.extract('(\d+)').astype(float)
movies['Duration'] = movies['Duration'].str.extract('(\d+)').astype(float)

In [ ]:
#Handle Votes column by removing commas and converting to float
movies['Votes'] = movies['Votes'].str.replace(',', '')
movies['Votes'] = pd.to_numeric(movies['Votes'], errors='coerce')

In [ ]:
# Fill missing values for numerical columns with the mean
movies[['Year', 'Duration', 'Votes']] = movies[['Year', 'Duration', 'Votes']].fillna(movies[['Year', 'Duration', 'Votes']].mean())

In [ ]:
# Combine relevant text features
movies['features'] = (movies['Genre'] + ' ' + movies['Director'] + ' ' +
                      movies['Actor 1'] + ' ' + movies['Actor 2'] + ' ' + movies['Actor 3'])

In [ ]:
# Vectorize text features
vectorizer = CountVectorizer()
X_text = vectorizer.fit_transform(movies['features'])

In [ ]:
 #Scale numerical features
scaler = StandardScaler()
X_numeric = scaler.fit_transform(movies[['Year', 'Duration', 'Votes']])

In [ ]:
# Combine text and numerical features
X = hstack([X_text, X_numeric])

In [ ]:
# Fill missing or empty strings in the 'Rating' column with a default value before converting to float
movies['Rating'] = movies['Rating'].replace('', np.nan).astype(float)
movies['Rating'] = movies['Rating'].fillna(movies['Rating'].mean())

In [ ]:
# Define target variable
y = movies['Rating']

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Train the model
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [ ]:
rr# Predictions
y_pred = model.predict(X_test)

In [ ]:
# Evaluation
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')
print(f'R^2 Score: {r2:.2f}')

Mean Squared Error: 1.90
R^2 Score: -0.98
